In [1]:
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.inspection import permutation_importance
from sklearn.metrics import r2_score
from scipy import stats
import matplotlib.pyplot as plt
import lightgbm as lgb
import math
import scipy
import conorm

In [2]:
def read_fasta(fasta_file):
    content = {}
    fas = open(fasta_file,'r')
    seq = ''
    last_entry = ''
    while True:
        line = fas.readline()
        if len(line)==0:
            content[last_entry]=seq
            break
        if '>' in line:
            if last_entry=='':
                line = line.rstrip()
                last_entry=line
            else:
                line = line.rstrip()
                content[last_entry]=seq
                last_entry=line
                seq=''
        else:
            line = line.rstrip()
            seq += line
    return content

def read_gff_complete(gff_file):
    import pandas as pd
    columns=['SeqID','Source','type','Start',
             'End','Score','Strand','Phase']
    df_raw = []
    retrieve_keys = []
    additional_info = []
    revised_info = []
    with open(gff_file, 'r') as f:
        while True:
            line = f.readline().rstrip()
            if line == '':
                f.close()
                break
            elif not line.startswith('#'):
                line = line.split('\t')
                df_raw.append(line[:-1])
                info_dict = {}
                for x in line[-1].split(';'):
                    if x.count('=') == 1:
                        k, v = x.split('=')
                        info_dict[k] = v
                additional_info.append(info_dict)
                retrieve_keys += list(info_dict.keys())
    retrieve_keys = list(np.unique(retrieve_keys))
    for t in additional_info:
        entry_list = []
        for k in retrieve_keys:
            if k in t:
                entry_list.append(t[k])
            else:
                entry_list.append('-')
        revised_info.append(entry_list)
    df = pd.DataFrame(df_raw,columns=columns)
    df[['Start','End']] = df[['Start','End']].astype(int)
    df[retrieve_keys] = revised_info
    return df.sort_values(by='Start').reset_index(drop=True)

def reverse_complement(s):
    rc_dict = {'A':'T','T':'A','C':'G','G':'C','a':'t','t':'a','c':'g','g':'c'}
    comp = [rc_dict[x] for x in list(s)]
    comp.reverse()
    return ''.join(comp)

### Feature compilation

In [3]:
tss = pd.read_csv('./data/LGBM/ref/MTB TSS+-35bp sequence.csv')
data = pd.read_csv('./data/RPKM Mtb.csv').set_index('Unnamed: 0')
sample = pd.read_excel('./data/sample/Sample Mtb.xlsx').set_index('Run')
gff = read_gff_complete('./ref/Mtb_GCF_000195955.2_ASM19595v2_genomic.gff')
mtb_g = read_fasta('./data/LGBM/ref/Mycobacterium_tuberculosis_H37Rv_genome_v4.fasta')
gseq = mtb_g[list(mtb_g.keys())[0]]
info_df = pd.read_csv("./data/LGBM/ref/TP and feature Mtb_LGBM format.csv").set_index('gene')
tf = pd.read_excel('./data/LGBM/ref/TF_Chipseq_ncomms6829_MOESM46_ESM.xlsx',sheet_name='same_as_sheet1')

In [4]:
TF_cutoff = 20
filtered = []
counts = tf.groupby('Regulator').count()
for x,c in zip(counts.index, counts['Gene']):
    if c>=TF_cutoff:
        filtered.append(tf[tf['Regulator']==x])
filtered = pd.concat(filtered)
regulators = filtered['Regulator'].unique()

In [5]:
gsize=len(gseq)
tf_table = []
tss_plus = np.sort(tss[tss['strand']=='+']['TSS.identifier'].values)
tss_minus = np.sort(tss[tss['strand']=='-']['TSS.identifier'].values)
c = 0
tp = []
nearest_tss = []
start_codon = []
genes = []
strand_sign = []
exp = []
gene_center = []
strand_list = []
for x in info_df.index:
    if x in gff['locus_tag'].values:
        start,end,strand = gff[gff['locus_tag']==x][['Start','End','Strand']].values[0]
        if strand == '+':
            start_codon.append(start)
            strand_sign.append(1)
            if start <100:
                gtss = tss_plus[-1]
            else:
                gtss = tss_plus[np.where((tss_plus-start)<10)[0][-1]]
        else:
            start_codon.append(end)
            gtss = tss_minus[np.where((tss_minus-end)>-10)[0][0]]
            strand_sign.append(-1)
        strand_list.append(strand)
        nearest_tss.append(gtss)
        genes.append(x)
        tp.append(info_df.loc[x,'TP'])
        exp.append(info_df.loc[x,'Median_logRPKM'])
        gene_center.append((start+end)/2)
    else:
        print(x)
tss_df = pd.DataFrame({'TP':tp,'TSS':nearest_tss,
                       'Start_codon':start_codon,
                       'Strand_sign':strand_sign,
                       'Expression':exp,'Center':gene_center,'Strand':strand_list},index=genes)


### Retieve promoter sequencing feature from MathFeature server

In [6]:
up = 60
down = 20

f = open('./data/LGBM/ref/tss_seq.fa','w')
pro_sequences = []

tsites = []
for i,(tsite,s) in enumerate(tss_df[['TSS','Strand']].values):
    if tsite not in tsites:
        tsites.append(tsite)
        g = tss_df.index[i]
        if s=='+':
            seq = gseq[(tsite-up):(tsite+down)]
        else:
            seq = reverse_complement(gseq[(tsite-down-1):(tsite+up-1)])
        pro_sequences.append(seq)
        f.write('>TSS_{}\n'.format(tsite))
        f.write('{}\n\n'.format(seq))
f.close()

In [7]:
f1 = pd.read_csv('./data/LGBM/ref/tss80_fourier_ext.csv')
f1['label'] = [int(x.split('_')[1]) for x in f1['nameseq'].values]
f1 = f1.set_index('label')
f1_cols = f1.columns[1:]
tss_df[['MF_{}'.format(x) for x in f1_cols]] = f1.loc[tss_df['TSS'].values][f1_cols].values

In [8]:
tss_df.shape

(3891, 26)

### TF binding feature compilation

In [9]:
basic_features_cols = info_df.columns[17:]
tss_df[basic_features_cols] = info_df.loc[tss_df.index,basic_features_cols].values;

/var/folders/1y/_2nhqmfd56s2k1hktst6p9b80000gn/T/ipykernel_70874/4258671928.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  tss_df[basic_features_cols] = info_df.loc[tss_df.index,basic_features_cols].values;
/var/folders/1y/_2nhqmfd56s2k1hktst6p9b80000gn/T/ipykernel_70874/4258671928.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  tss_df[basic_features_cols] = info_df.loc[tss_df.index,basic_features_cols].values;
/var/folders/1y/_2nhqmfd56s2k1hktst6p9b80000gn/T/ipykernel_70874/4258671928.py:21: PerformanceWarning: DataF

In [13]:
len(basic_features_cols)

101

In [12]:
tss_df.shape

(3891, 127)

In [11]:
tss_df.columns[8:]

Index(['MF_median', 'MF_maximum', 'MF_minimum', 'MF_peak',
       'MF_none_levated_peak', 'MF_sample_standard_deviation',
       'MF_population_standard_deviation', 'MF_percentile15',
       'MF_percentile25', 'MF_percentile50',
       ...
       'l_beta', 'u_beta', 'beta_max_span', 'M', 'l_M', 'u_M', 'M_span',
       'Vulnerability.Index', 'VI.Lower.Bound', 'VI.Upper.Bound'],
      dtype='object', length=119)

In [80]:
X = tss_df[tss_df.columns[8:]].values
X[np.isnan(X)]=0
Y = np.log2(tss_df['TP'].values).reshape(-1,1) # use log2-transformed TP instead

In [81]:
def min_max(data):
    normd = (data-data.min(axis=0)[np.newaxis,:])/(data.max(axis=0)-data.min(axis=0))[np.newaxis,:]
    return normd

In [82]:
X_norm = min_max(X)
Y_norm = min_max(Y).flatten()

In [152]:
# initiate importance dict
importance_dict = pd.DataFrame({'feature_name':tss_df.columns[8:]})

# record fit stats
R = []
MSE = []

# iterative training
iter_n = 100
test_size = 0.4
num_leaves = 30
learning_rate = 0.015
feature_fraction=0.8
n_estimators = 200
max_depth=15
max_bin=10
bagging_fraction=1


for i in range(0,iter_n):
    X_train, X_test, y_train, y_test = train_test_split(X_norm, Y_norm, 
                                                        test_size=test_size, 
                                                        random_state=i)
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)
    gbm = lgb.LGBMRegressor(objective='regression', reg_lambda=1,
                            max_bin=max_bin,
                            num_leaves=num_leaves,
                            learning_rate=learning_rate, 
                            verbose=0,
                            n_estimators=n_estimators,
                            max_depth=max_depth,
                            n_jobs=16)
    
    gbm.fit(X_train, y_train, 
            eval_set=[(X_test, y_test)], 
            eval_metric='l2')
    
    y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration_)
    y_train_eval = gbm.predict(X_train, num_iteration=gbm.best_iteration_)
    feature_importance = gbm.feature_importances_
    
    importance_dict_iter = pd.DataFrame({'feature_name':tss_df.columns[8:],
                                         i:feature_importance})
    importance_dict = pd.merge(importance_dict,importance_dict_iter,how='left',on='feature_name')

    R.append([scipy.stats.pearsonr(y_test, y_pred)[0],
              scipy.stats.pearsonr(y_train, y_train_eval)[0]])
    MSE.append(mean_squared_error(y_test, y_pred))

In [159]:
tss_df.loc['Rv0049']

TP                     0.635053
TSS                       52810
Start_codon               52831
Strand_sign                   1
Expression             8.756984
                         ...   
u_M                       0.582
M_span                    0.417
Vulnerability.Index      -0.696
VI.Lower.Bound           -2.858
VI.Upper.Bound            1.664
Name: Rv0049, Length: 127, dtype: object

In [154]:
x = importance_dict['feature_name'].values[np.flip(np.argsort(np.mean(importance_dict.values[:,1:],axis=1)))]
x

array(['operon_length', 'basePercent_GC', 'Activator.number', 'width',
       'basePercent_G', 'MF_maximum', 'A3s', 'AA_C', 'AA_A',
       'maxCov.TSS.site', 'distance_to_ori', 'AA_R', 'MF_amplitude',
       'X2.5..quantile.κ', 'u_beta', 'l_gamma', 'AA_S', 'AA_Y',
       'basePercent_T', 'basePercent_CT', 'AA_L', 'Repressor.number',
       'X97.5..quantile.κ', 'Nc', 'AA_H', 'Rep.Direction', 'MF_minimum',
       'basePercent_C', 'MF_kurtosis', 'MF_percentile25', 'MF_peak',
       'MF_sample_standard_deviation', 'AA_D', 'MF_none_levated_peak',
       'AA_E', 'C3s', 'maxCov.TSS.MeanCoverage', 'MF_percentile15',
       'MF_skewness', 'AA_G', 'Geometric.mean.ω', 'AA_P', 'AA_F',
       'maxCov.TSS.distance2gene', 'G3s', 'MF_median', 'Aromo', 'GC3s',
       'MF_percentile75', 'AA_I', 'AA_V', 'AA_K', 'AA_M', 'AA_Q',
       'n_guides', 'T3s', 'Mean.Pr.ω.1.', 'l_M', 'X97.5..quantile.θ',
       'u_H', 'AA_N', 'X2.5..quantile.geometric.mean.ω', 'basePercent_A',
       'MF_interquartile_range', 'AA

In [155]:
importance_dict.to_csv('./data/LGBM/ref/LGBM feature importance.csv')

In [156]:
np.savetxt("./data/LGBM/ref/Prediction_LGBMR_TP_r.txt", np.array(R), fmt='%f')
np.savetxt("./data/LGBM/ref/Prediction_LGBMR_TP_mse.txt", np.array(MSE), fmt='%f')

In [109]:
tss_df[['MF_{}'.format(x) for x in f1_cols]].to_excel('./data/LGBM/ref/MathFeatureBinaryFourier.xlsx')

In [116]:
importance_dict['feature_name']

0                 MF_median
1                MF_maximum
2                MF_minimum
3                   MF_peak
4      MF_none_levated_peak
               ...         
114                     u_M
115                  M_span
116     Vulnerability.Index
117          VI.Lower.Bound
118          VI.Upper.Bound
Name: feature_name, Length: 119, dtype: object